In [ ]:
# Third-party
import astropy.coordinates as coord
import astropy.units as u
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from scipy.interpolate import InterpolatedUnivariateSpline

from pyia import GaiaData
from ezpadova import parsec
from imf.brokenimf import new_kroupa_mass_distribution # borrowed from AMUSE

In [ ]:
g = GaiaData('../data/products/gaia-selected.fits')

In [ ]:
ms = new_kroupa_mass_distribution(100000)

In [ ]:
plt.hist(ms.value, bins=np.logspace(-2, 2, 100));
plt.xscale('log')
plt.yscale('log')

In [ ]:
_iso_cache = {}

In [ ]:
import copy

Zsun = 0.0152

class PARSECIsochrone:
    
    _label_to_name = {0: 'PMS',
                      1: 'MS',
                      2: 'SGB',
                      3: 'RGB',
                      4: 'CHeB1',
                      5: 'CHeB2',
                      6: 'CHeB3',
                      7: 'EAGB',
                      8: 'TPAGB',
                      9: 'WD'}
    
    def __init__(self, filename=None, tbl=None, age=None, phot=None, 
                 fe_h=None, Z=None, **kwargs):
        
        if filename is not None:
            raise NotImplementedError()
            
        elif tbl is not None:
            self.tbl = tbl
            
        else:
            if age is None or phot is None or (fe_h is None and Z is None):
                raise ValueError('If no filename is specified, you must pass '
                                 'in age, photometry name, and metallicity '
                                 'information. See `parsec.get_one_isochrone()`')
        
            if fe_h is not None and Z is not None:
                raise ValueError('You can only specify one of Z or [Fe/H] (fe_h) '
                                 'to specify the metallicity')
                
            if hasattr(age, 'unit'):
                age = age.to(u.yr).value
            
            if isinstance(phot, str):
                phot = [phot]
                
            if fe_h is not None:
                Z = Zsun * 10**fe_h
            
            iso_tbls = []
            for ph in phot:
                cache_key = (age, Z, ph)
                if cache_key in _iso_cache: # isochrone in cache
                    iso = _iso_cache[cache_key]
                
                else: # download isochrone, not in cache
                    iso = parsec.get_one_isochrone(age, Z, phot=ph)
                    iso = iso.to_astropy_table()
                    _iso_cache[cache_key] = iso
                    
                iso_tbls.append(iso)
                
            self.tbl = self._merge_isos(iso_tbls)
        
    def _merge_isos(self, isos):
        if len(isos) == 0:
            return
        
        master_iso = isos[0]

        for iso in isos[1:]:
            if len(iso) != len(master_iso):
                raise ValueError('Lengths must match to merge isochrones.')

            for c in iso.colnames:
                if c in master_iso.colnames:
                    continue

                master_iso[c] = iso[c]

        return master_iso
    
    ##########################################################################
    # Python internal
    #
    
    def __getattr__(self, name):
        # to prevent recursion errors:
        # nedbatchelder.com/blog/201010/surprising_getattr_recursion.html
        if name in ['tbl']:
            raise AttributeError()
            
        elif name not in self.tbl.colnames:
            return self.__getattribute__(name)
            
        return self.tbl[name]

    def __dir__(self):
        return super().__dir__() + self.tbl.colnames
    
    def __getitem__(self, slc):
        if isinstance(slc, int):
            slc = slice(slc, slc+1)
        return type(self)(tbl=self.tbl[slc])

    def __len__(self):
        return len(self.tbl)
    
    
    ##########################################################################
    # Custom
    #
    
    @property
    def stage_names(self):
        if not hasattr(self, '_stage_names'):
            self._stage_names = np.array([self._label_to_name[l] for l in self.label])
        return self._stage_names

In [ ]:
iso = PARSECIsochrone(age=150 * u.Myr,
                      fe_h=-1.,
                      phot=['gaiaDR2', 'decam'])

In [ ]:
ms_iso = iso[iso.stage_names == 'MS']

In [ ]:
interp_G = InterpolatedUnivariateSpline(iso.Mini, iso.Gmag, ext=3)
interp_BPRP = InterpolatedUnivariateSpline(iso.Mini, iso.G_BPmag-iso.G_RPmag, ext=3)

In [ ]:
mini_grid = np.logspace(np.log10(0.08), np.log10(5), 1024)

fig, axes = plt.subplots(1, 2, figsize=(10, 5), sharex=True)

axes[0].plot(iso.Mini, iso.Gmag)
axes[0].set_ylim(15, -5)
axes[0].plot(mini_grid, interp_G(mini_grid), marker='')

axes[1].plot(iso.Mini, iso.G_BPmag-iso.G_RPmag)
axes[1].plot(mini_grid, interp_BPRP(mini_grid), marker='')

In [ ]:
Gs = interp_G(ms.value)
BPRPs = interp_BPRP(ms.value)
DM = coord.Distance(30*u.kpc).distmod.value

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(4, 6))
ax.plot(BPRPs, Gs + DM, 
        marker='.', ls='none', alpha=0.3, color='k')

ax.scatter(g.BP0 - g.RP0, g.G0, zorder=-100, 
           alpha=0.5, color='tab:green')

nodes = np.array([[-0.25, 15.8],
                  [0.12, 19.8],
                  [-0.2, 20.1],
                  [-0.5, 16.1],
                  [-0.25, 15.8]])
ax.plot(nodes[:, 0], nodes[:, 1], marker='', color='tab:blue', lw=2)
cmd_path = mpl.patches.Path(nodes)

ax.set_ylim(21, 12)
ax.set_xlim(-1, 2)

In [ ]:
sim_mask = cmd_path.contains_points(np.vstack((BPRPs, Gs + DM)).T)
frac = len(g) / sim_mask.sum()

In [ ]:
frac * ms.sum()